## Setup

In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as sm
import torch
import torch.nn as nn
from sklearn.preprocessing import OneHotEncoder
import plotnine as p9
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

## Data

In [ ]:
url = 'https://raw.githubusercontent.com/Middleton-Lab/abdData/main/inst/extdata/datasets/18/18e4MoleRatLayabouts.csv'

# Read the CSV file with a specific delimiter and encoding
df = pd.read_csv(url, delimiter = ',', encoding = 'utf-8')
df.rename(columns = {'ln.energy': 'log_energy', 'ln.mass': 'log_mass'},
          inplace = True)

# Display the first few rows
df.head()

In [ ]:
p = (p9.ggplot() +
    p9.geom_point(df,
                  p9.aes(x = 'log_mass', y = 'log_energy', color = 'caste'),
                  size = 3, alpha = 0.5) +
    p9.scale_color_manual(values = ['firebrick', 'darkblue'], name = "Group") +
    p9.labs(x = 'log Mass', y = 'log Energy') +
    p9.theme_classic() +
    p9.theme(figure_size = (8, 5)))
p.show()

## Regression of caste and log mass on log energy using `statsmodels`

In [4]:
model_ols = sm.ols("log_energy ~ caste + log_mass", data = df).fit()
# model_ols.model.exog

In [ ]:
print(model_ols.summary())

## R for comparison

```
Call:
lm(formula = ln.energy ~ caste + ln.mass, data = D)

Residuals:
     Min       1Q   Median       3Q      Max
-0.73388 -0.19371  0.01317  0.17578  0.47673

Coefficients:
            Estimate Std. Error t value Pr(>|t|)
(Intercept) -0.09687    0.94230  -0.103   0.9188
casteworker  0.39334    0.14611   2.692   0.0112 *
ln.mass      0.89282    0.19303   4.625 5.89e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.2966 on 32 degrees of freedom
Multiple R-squared:  0.409,	Adjusted R-squared:  0.3721
F-statistic: 11.07 on 2 and 32 DF,  p-value: 0.0002213
```

## Train the model

In [ ]:
# Numeric feature
x1 = df['log_mass'].values.reshape(-1, 1)
x1.shape

Encoding categorical variables

In [ ]:
# Categorical feature
# Encode the categorical feature
encoder = OneHotEncoder(sparse_output = False)
x2_encoded = encoder.fit_transform(df['caste'].values.reshape(-1, 1))

# Combine features
X = np.hstack((x1, x2_encoded))
X.shape

Reshaping the outcome variable

In [ ]:
# Target variable
y = df['log_energy'].values.reshape(-1, 1)
y.shape

Convert to PyTorch tensors

In [9]:
X_tensor = torch.tensor(X, dtype = torch.float32)
y_tensor = torch.tensor(y, dtype = torch.float32)

Define the model

In [10]:
class MultipleRegressionModel(nn.Module):
    def __init__(self):
        super(MultipleRegressionModel, self).__init__()
        self.linear = nn.Linear(3, 1)  # Input dimension is 3 (x1, x2_encoded),
                                       # output dimension is 1

    def forward(self, x):
        return self.linear(x)

model = MultipleRegressionModel()

# Step 3: Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.001)

Train the model

In [ ]:
num_epochs = 1000
loss_values = []  # List to store loss values

for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # zero grad before new step
    optimizer.zero_grad()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    # Store loss value
    loss_values.append(np.log(loss.item()))

Plot loss by epoch

In [ ]:
p2 = (p9.ggplot(p9.aes(x = range(1, num_epochs + 1), y = loss_values)) +
      p9.geom_line(size = 1, color = 'firebrick') +
      p9.labs(x = 'Epoch', y = 'log Loss') +
      p9.theme_classic() +
      p9.theme(figure_size = (8, 5)))
p2.show()

## Predictions

OLS model

In [13]:
pred_ols = model_ols.predict(df)
df['predicted_ols'] = pred_ols

NN model

In [14]:
model.eval()
predicted = model(X_tensor).detach().numpy()
df['predicted_NN'] = predicted

In [ ]:
p3 = p + [p9.geom_point(p9.aes(x ='log_mass',
                                y = 'predicted_NN',
                                color = 'caste'),
                        data = df, shape = 'x', size = 4),
          p9.geom_point(p9.aes(x ='log_mass', 
                                y = 'predicted_ols',
                                color = 'caste'),
                        data = df, shape = 'o', size = 4)]
p3.show()

## Interaction model

In [18]:
# df2 has the model matrix hard coded
df2 = pd.read_csv('molerats.csv')

# Outcome
y = df2['log_energy'].values.reshape(-1, 1)

# Model matrix for interaction model with intercept
X = df2.iloc[:, 0:4].values

# Convert to PyTorch tensors
X_tensor = torch.tensor(X, dtype = torch.float32)
y_tensor = torch.tensor(y, dtype = torch.float32)

In [ ]:
# Define the model
class MultipleRegressionModel(nn.Module):
    def __init__(self):
        super(MultipleRegressionModel, self).__init__()
        self.linear = nn.Linear(4, 1)  # Input dimension is 4, output dimension is 1

    def forward(self, x):
        return self.linear(x)

model = MultipleRegressionModel()

# Define the loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# Train the model
num_epochs = 1000
loss_values = []  # List to store loss values

for epoch in range(num_epochs):
    model.train()
    
    # Forward pass
    outputs = model(X_tensor)
    loss = criterion(outputs, y_tensor)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # zero grad before new step
    optimizer.zero_grad()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}')
    
    # Store loss value
    loss_values.append(np.log(loss.item()))

p4 = (p9.ggplot(p9.aes(x = range(1, num_epochs + 1), y = loss_values)) +
      p9.geom_line(size = 1, color = 'firebrick') +
      p9.labs(x = 'Epoch', y = 'log Loss') +
      p9.theme_classic() +
      p9.theme(figure_size = (8, 5)))
p4.show()

In [ ]:
model.eval()
predicted = model(X_tensor).detach().numpy()
df['predicted_NN2'] = predicted

p3 + [p9.geom_point(p9.aes(x = 'log_mass', y = 'predicted_NN2',
                            color = 'caste'),
                    data = df, shape = '+', size = 4)]